# checks for matches between issns

In [4]:
import requests
import pandas as pd

def fetch_dois_by_issn(issn):
    api_url = f'https://api.crossref.org/journals/{issn}/works'
    response = requests.get(api_url)

    if response.status_code == 200:
        data = response.json()
        return data.get('message', {}).get('items', [])
    else:
        print(f"Error fetching DOIs for ISSN {issn}. Status code: {response.status_code}")
        return []

def create_dataframe(dois):
    rows = []

    for doi in dois:
        title = doi.get('title', [''])[0]
        journal_title = doi.get('container-title', [''])[0]
        publication_year = doi.get('published-print', {}).get('date-parts', [[None]])[0][0]

        rows.append({'DOI': doi['DOI'], 'Title': title, 'Journal Title': journal_title, 'Publication Year': publication_year})

    return pd.DataFrame(rows)

def find_duplicates(df):
    duplicates = df[df.duplicated(subset=['Title', 'Journal Title', 'Publication Year'], keep=False)]
    return duplicates

def main(issn1, issn2):
    # Replace these with your actual ISSNs
    #issn1 = '1234-5678'
    #issn2 = '9876-5432'

    # Fetch DOIs for the given ISSNs
    dois_issn1 = fetch_dois_by_issn(issn1)
    dois_issn2 = fetch_dois_by_issn(issn2)

    # Create dataframes for each ISSN
    df1 = create_dataframe(dois_issn1)
    df2 = create_dataframe(dois_issn2)

    # Merge dataframes on common columns (DOI, Title, Journal Title, Publication Year)
    merged_df = pd.merge(df1, df2, on=['Title', 'Journal Title', 'Publication Year'], how='inner', suffixes=('_issn1', '_issn2'))

    # Find and display duplicates
    duplicates = find_duplicates(merged_df)
    print("Duplicates based on matching Title, Journal Title, and Publication Year:")
    print(duplicates)




In [ ]:
main()

In [9]:
pip install fuzzywuzzy

In [37]:
import requests
import pandas as pd
from fuzzywuzzy import fuzz

def fetch_metadata_by_issn(issn):
    api_url = f'https://api.crossref.org/works?filter=issn:{issn}&select=DOI,title,container-title,issued'
    response = requests.get(api_url)

    if response.status_code == 200:
        data = response.json()
        return data.get('message', {}).get('items', [])
    else:
        print(f"Error fetching metadata for ISSN {issn}. Status code: {response.status_code}")
        return []

def create_dataframe(metadata):
    rows = []

    for entry in metadata:
        doi = entry.get('DOI', '')
        title = entry.get('title', '')
        journal_title = entry.get('container-title', [''])[0]
        publication_date = entry.get('issued', {}).get('date-parts', [[None]])[0][0]

        rows.append({'DOI': doi, 'Title': title, 'Journal Title': journal_title, 'Publication Date': publication_date})

    return pd.DataFrame(rows)

def find_duplicates(df):
    df['Title Cleaned'] = df['Title'].apply(lambda x: x[0].lower() if x and pd.notna(x[0]) else x)

    # Use the fuzzywuzzy library to calculate Levenshtein distance
    df['Levenshtein Ratio'] = df.groupby('Journal Title')['Title Cleaned'].apply(lambda x: x.apply(lambda y: fuzz.ratio(x.iloc[0], y) if pd.notna(x.iloc[0]) else 0))

    # You can adjust the threshold as needed
    duplicates = df[df['Levenshtein Ratio'] > 50]  # Change 90 to the desired threshold

    return duplicates

def main(issn):
    # Replace this with your actual ISSN
    #issn = '1875-6883'

    # Fetch metadata for the given ISSN
    metadata = fetch_metadata_by_issn(issn)

    # Create dataframe
    df = create_dataframe(metadata)

    # Find and display duplicates based on Levenshtein distance
    duplicates = find_duplicates(df)
    print("Duplicates based on Levenshtein distance in titles:")
    print(duplicates)




In [38]:
issn = "18756883"
main(issn)

Duplicates based on Levenshtein distance in titles:
                            DOI  \
0  10.1080/18756891.2013.865403   

                                               Title  \
0  [A New Adaptive and Self Organizing Fuzzy Poli...   

                                       Journal Title  Publication Date  \
0  International Journal of Computational Intelli...              2014   

                                       Title Cleaned  Levenshtein Ratio  
0  a new adaptive and self organizing fuzzy polic...                100  


<ipython-input-37-fc85d4014cdb>:33: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df['Levenshtein Ratio'] = df.groupby('Journal Title')['Title Cleaned'].apply(lambda x: x.apply(lambda y: fuzz.ratio(x.iloc[0], y) if pd.notna(x.iloc[0]) else 0))


In [41]:
import pandas as pd
from fuzzywuzzy import fuzz
import re

def read_excel_file(file_path):
    # Read data from Excel file
    df = pd.read_excel(file_path)
    return df

def clean_title(title):
    # Remove specific characters and square brackets and their contents
    if isinstance(title, str):
        title = re.sub(r'\[[^\]]*\]', '', title)
        title = re.sub('[^a-zA-Z0-9\s]', '', title)
    elif isinstance(title, list):
        # Join the list elements into a string
        title = ' '.join(map(str, title))
        title = re.sub(r'\[[^\]]*\]', '', title)
        title = re.sub('[^a-zA-Z0-9\s]', '', title)
    return title

def create_dataframe(df):
    rows = []

    for index, row in df.iterrows():
        doi = row.get('DOI', '')  # Adjust column name based on your Excel file
        title = row.get('title', '')  # Adjust column name based on your Excel file
        title = clean_title(title)  # Clean the title
        journal_title = row.get('container-title', '')  # Adjust column name based on your Excel file
        publication_date = row.get('issued', None)  # Adjust column name based on your Excel file

        rows.append({'DOI': doi, 'Title': title, 'Journal Title': journal_title, 'Publication Date': publication_date})

    return pd.DataFrame(rows)

def find_duplicates(df):
    df['Title Cleaned'] = df['Title'].apply(lambda x: x[0].lower() if isinstance(x, list) and x and pd.notna(x[0]) else x)

    # Use the fuzzywuzzy library to calculate Levenshtein distance
    df['Levenshtein Ratio'] = df.groupby('Journal Title', group_keys=False)['Title Cleaned'].apply(lambda x: x.apply(lambda y: fuzz.ratio(x.iloc[0], y[0]) if isinstance(x.iloc[0], list) and pd.notna(x.iloc[0]) else 0))

    # You can adjust the threshold as needed
    duplicates = df[df['Levenshtein Ratio'] > 20]  # Change 90 to the desired threshold

    return duplicates

def main(file_path):
    # Read data from Excel file
    df = read_excel_file(file_path)

    # Create dataframe
    df_cleaned = create_dataframe(df)

    # Find and display duplicates based on Levenshtein distance
    duplicates = find_duplicates(df_cleaned)
    print("Duplicates based on Levenshtein distance in titles:")
    print(duplicates)

# Replace 'your_excel_file.xlsx' with the path to your Excel file
excel_file_path = '/content/drive/MyDrive/Colab Notebooks/Crossref_notebooks/all_titles_for_ISSN_18756883.xlsx'
main(excel_file_path)


Duplicates based on Levenshtein distance in titles:
Empty DataFrame
Columns: [DOI, Title, Journal Title, Publication Date, Title Cleaned, Levenshtein Ratio]
Index: []


This works and saves out an excel fie.

In [44]:
import pandas as pd

def read_excel_file(file_path):
    # Read data from Excel file
    df = pd.read_excel(file_path)
    return df

def clean_title(title):
    # Remove specific characters and square brackets and their contents
    if isinstance(title, str):
        title = re.sub(r'\[[^\]]*\]', '', title)
        title = re.sub('[^a-zA-Z0-9\s]', '', title)
    elif isinstance(title, list):
        # Join the list elements into a string
        title = ' '.join(map(str, title))
        title = re.sub(r'\[[^\]]*\]', '', title)
        title = re.sub('[^a-zA-Z0-9\s]', '', title)
    return title

def create_dataframe(df):
    rows = []

    for index, row in df.iterrows():
        doi = row.get('DOI', '')  # Adjust column name based on your Excel file
        title = row.get('title', '')  # Adjust column name based on your Excel file
        title = clean_title(title)  # Clean the title
        journal_title = row.get('container-title', '')  # Adjust column name based on your Excel file
        publication_date = row.get('issued', None)  # Adjust column name based on your Excel file

        rows.append({'DOI': doi, 'Title': title, 'Journal Title': journal_title, 'Publication Date': publication_date})

    return pd.DataFrame(rows)

def find_duplicates(df):
    duplicates = df[df.duplicated(subset=['Title'], keep=False)]
    return duplicates

def save_to_excel(df, file_path):
    df.to_excel(file_path, index=False)
    print(f"DataFrame saved to {file_path}")

def main(file_path):
    # Read data from Excel file
    df = read_excel_file(file_path)

    # Create dataframe
    df_cleaned = create_dataframe(df)

    # Find and display duplicates based on exact title matching
    duplicates = find_duplicates(df_cleaned)
    print("Duplicates based on exact title matching:")
    print(duplicates)

    # Save duplicates to a new Excel file
    save_to_excel(duplicates, '/content/drive/MyDrive/Colab Notebooks/Crossref_notebooks/duplicates_output.xlsx')

# Replace 'your_excel_file.xlsx' with the path to your Excel file
excel_file_path = '/content/drive/MyDrive/Colab Notebooks/Crossref_notebooks/all_titles_for_ISSN_18756883.xlsx'
main(excel_file_path)


Duplicates based on exact title matching:
                                DOI  \
0           10.2991/jnmp.2008.1.3.1   
5          10.2991/ijcis.2010.3.2.7   
6     10.1080/18756891.2010.9727690   
11    10.1080/18756891.2011.9727879   
12        10.2991/ijcis.2011.4.6.22   
...                             ...   
2110   10.1080/18756891.2014.891369   
2111   10.1080/18756891.2014.963976   
2112  10.1080/18756891.2011.9727766   
2113       10.2991/ijcis.2011.4.1.8   
2114   10.1080/18756891.2013.808426   

                                                  Title  \
0                                                         
5                       A nave glance at Soft Computing   
6                       A nave glance at Soft Computing   
11    A Calibration Method for A Linear Structured L...   
12    A Calibration Method for A Linear Structured L...   
...                                                 ...   
2110                                                      
2111             